# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = os.path.join("..","WeatherPy","output_data","cities.csv")
cities_df = pd.read_csv(file_path)
cities_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Busselton,14.0,AU,1.603858e+09,32.0,-33.65,115.33,75.99,7.83
1,1,Vardø,75.0,NO,1.603858e+09,93.0,70.37,31.11,37.40,14.99
2,2,Vaini,20.0,TO,1.603858e+09,74.0,-21.20,-175.20,78.80,9.17
3,3,Yellowknife,40.0,CA,1.603858e+09,72.0,62.46,-114.35,14.00,6.93
4,4,Port Alfred,54.0,ZA,1.603858e+09,83.0,-33.59,26.89,63.34,11.34
...,...,...,...,...,...,...,...,...,...,...
565,565,Rumonge,100.0,BI,1.603858e+09,69.0,-3.97,29.44,73.89,3.13
566,566,Campbell River,90.0,CA,1.603858e+09,100.0,50.02,-125.24,44.60,2.24
567,567,Lanigan,64.0,CA,1.603858e+09,84.0,51.85,-105.03,37.89,14.07
568,568,Merritt Island,40.0,US,1.603858e+09,88.0,28.54,-80.67,80.01,5.82


In [3]:
#cities_df.query("Country == 'MX'")

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
55,55,Cabo San Lucas,40.0,MX,1.603858e+09,69.0,22.89,-109.91,73.40,5.82
102,102,Atotonilquillo,5.0,MX,1.603858e+09,60.0,20.40,-103.13,68.00,3.36
124,124,San Quintín,0.0,MX,1.603858e+09,54.0,30.48,-115.95,60.85,1.19
136,136,Guerrero Negro,0.0,MX,1.603858e+09,37.0,27.98,-114.06,62.49,13.71
149,149,Tomatlán,92.0,MX,1.603858e+09,90.0,19.93,-105.25,77.61,1.86
161,161,Acaponeta,14.0,MX,1.603858e+09,82.0,22.50,-105.36,76.10,1.19
196,196,San Miguel de Cozumel,98.0,MX,1.603858e+09,66.0,20.51,-86.95,80.01,8.01
273,273,Pochutla,70.0,MX,1.603858e+09,94.0,15.74,-96.47,77.00,3.18
288,288,Niltepec,43.0,MX,1.603858e+09,85.0,16.57,-94.62,76.17,1.81
291,291,Badiraguato,5.0,MX,1.603858e+09,28.0,25.37,-107.52,73.40,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with api key
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# Add layer to the map
fig.add_layer(heat_layer)

# Show map
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
# Conditions that I wanted to filter
temps_high = cities_df["Max Temp"] > 70 # 21 (C) - 294 (K) - 70 (F)
temps_low = cities_df["Max Temp"] < 80 # 27 (C) - 300 (K) - 80 (F)
humidity_high = cities_df["Humidity"] > 68
humidity_low = cities_df["Humidity"] < 95
cloudiness_low = cities_df["Cloudiness"] < 6.0
windspeed_low = cities_df["Wind Speed"] < 10.0

In [30]:
# Filter DataFrame and populate hotel_df
hotel_df = cities_df.loc[temps_high & temps_low & humidity_low & humidity_high & cloudiness_low & windspeed_low]
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
108,108,Antalaha,1.0,MG,1.603858e+09,71.0,-14.90,50.28,75.72,5.21
128,128,Grand Gaube,1.0,MU,1.603858e+09,72.0,-20.01,57.66,75.99,3.00
134,134,Sambava,2.0,MG,1.603858e+09,73.0,-14.27,50.17,75.51,7.70
167,167,Damietta,0.0,EG,1.603858e+09,88.0,31.42,31.81,73.40,6.93
217,217,Souillac,4.0,MU,1.603858e+09,72.0,-20.52,57.52,75.99,3.00
257,257,Hobyo,4.0,SO,1.603858e+09,75.0,5.35,48.53,79.97,6.38
285,285,Masuguru,0.0,TZ,1.603858e+09,72.0,-11.37,38.42,74.80,3.62
384,384,Nacala,0.0,MZ,1.603858e+09,77.0,-14.54,40.67,76.55,3.09
426,426,Maxixe,3.0,MZ,1.603858e+09,86.0,-23.86,35.35,73.45,8.19
476,476,San Jose de Guanipa,0.0,VE,1.603858e+09,84.0,8.89,-64.17,75.94,8.81


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# Set the Hotel Name column into DataFrame - empty for now
hotel_df["Hotel Name"] = ""

# Set up some variables
target_coordinates = "22.89,-109.91"
target_radius = 5000
target_search = "Hotel"

hotel_df

<ipython-input-31-121e19a2bd4b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
108,108,Antalaha,1.0,MG,1.603858e+09,71.0,-14.90,50.28,75.72,5.21,
128,128,Grand Gaube,1.0,MU,1.603858e+09,72.0,-20.01,57.66,75.99,3.00,
134,134,Sambava,2.0,MG,1.603858e+09,73.0,-14.27,50.17,75.51,7.70,
167,167,Damietta,0.0,EG,1.603858e+09,88.0,31.42,31.81,73.40,6.93,
217,217,Souillac,4.0,MU,1.603858e+09,72.0,-20.52,57.52,75.99,3.00,
257,257,Hobyo,4.0,SO,1.603858e+09,75.0,5.35,48.53,79.97,6.38,
285,285,Masuguru,0.0,TZ,1.603858e+09,72.0,-11.37,38.42,74.80,3.62,
384,384,Nacala,0.0,MZ,1.603858e+09,77.0,-14.54,40.67,76.55,3.09,
426,426,Maxixe,3.0,MZ,1.603858e+09,86.0,-23.86,35.35,73.45,8.19,
476,476,San Jose de Guanipa,0.0,VE,1.603858e+09,84.0,8.89,-64.17,75.94,8.81,


In [32]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"



In [33]:
# Iterate over the hotels_df
responses = []
for index, row in hotel_df.iterrows():
    # set up parameters
    params = {
        "location": f"{row['Lat']},{row['Lng']}",
        "radius": target_radius,
        "keyword": target_search,
        "key": g_key
    }
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    print(response)
    response = response.json()
    
    #add response to responses to treat data later
    responses.append(response)
    try:
        # Add the hotel name to the DataFrame
        hotel_df.loc[index,"Hotel Name"]= response["results"][0]["name"]
        #print(response["results"][0]["name"])
    except:
        # If not found just put an empty string
        hotel_df.loc[index,"Hotel Name"] = pd.NA
        #print("Error with the hotel")




<Response [200]>


c:\users\luis\dropbox\personal\dataanalytics\bootcamp\bootcamp\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [34]:
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
108,108,Antalaha,1.0,MG,1.603858e+09,71.0,-14.90,50.28,75.72,5.21,Belle rose beach Hotel
128,128,Grand Gaube,1.0,MU,1.603858e+09,72.0,-20.01,57.66,75.99,3.00,LUX Grand Gaube Resort & Villas
134,134,Sambava,2.0,MG,1.603858e+09,73.0,-14.27,50.17,75.51,7.70,Hôtel Las Palmas
167,167,Damietta,0.0,EG,1.603858e+09,88.0,31.42,31.81,73.40,6.93,فندق المنشي دمياط
217,217,Souillac,4.0,MU,1.603858e+09,72.0,-20.52,57.52,75.99,3.00,SkylineInn Hotel
257,257,Hobyo,4.0,SO,1.603858e+09,75.0,5.35,48.53,79.97,6.38,Osmani Hotel
285,285,Masuguru,0.0,TZ,1.603858e+09,72.0,-11.37,38.42,74.80,3.62,<NA>
384,384,Nacala,0.0,MZ,1.603858e+09,77.0,-14.54,40.67,76.55,3.09,Nacala Plaza Business Design Hotel
426,426,Maxixe,3.0,MZ,1.603858e+09,86.0,-23.86,35.35,73.45,8.19,Hotel Casa Do Capitão
476,476,San Jose de Guanipa,0.0,VE,1.603858e+09,84.0,8.89,-64.17,75.94,8.81,HOTEL 283 SUITES


In [35]:
# Clean the DataFrame
narrowed_city_df = hotel_df.dropna(subset=["Hotel Name"])
narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
108,108,Antalaha,1.0,MG,1.603858e+09,71.0,-14.90,50.28,75.72,5.21,Belle rose beach Hotel
128,128,Grand Gaube,1.0,MU,1.603858e+09,72.0,-20.01,57.66,75.99,3.00,LUX Grand Gaube Resort & Villas
134,134,Sambava,2.0,MG,1.603858e+09,73.0,-14.27,50.17,75.51,7.70,Hôtel Las Palmas
167,167,Damietta,0.0,EG,1.603858e+09,88.0,31.42,31.81,73.40,6.93,فندق المنشي دمياط
217,217,Souillac,4.0,MU,1.603858e+09,72.0,-20.52,57.52,75.99,3.00,SkylineInn Hotel
257,257,Hobyo,4.0,SO,1.603858e+09,75.0,5.35,48.53,79.97,6.38,Osmani Hotel
384,384,Nacala,0.0,MZ,1.603858e+09,77.0,-14.54,40.67,76.55,3.09,Nacala Plaza Business Design Hotel
426,426,Maxixe,3.0,MZ,1.603858e+09,86.0,-23.86,35.35,73.45,8.19,Hotel Casa Do Capitão
476,476,San Jose de Guanipa,0.0,VE,1.603858e+09,84.0,8.89,-64.17,75.94,8.81,HOTEL 283 SUITES
521,521,Moroni,0.0,KM,1.603858e+09,71.0,-11.70,43.26,77.59,3.60,Golden Tulip Grande Comore Moroni Resort & Spa


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))